## Loading the Data

In [1]:
from datasets import load_dataset
dataset = load_dataset('billingsmoore/tibetan-to-english-translation-dataset', split='train')

Using the latest cached version of the dataset since billingsmoore/tibetan-to-english-translation-dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/j/.cache/huggingface/datasets/billingsmoore___tibetan-to-english-translation-dataset/default/0.0.0/489c8f300991c7fafd496c8f91cf451d34192d8e (last modified on Sat Feb  1 09:10:10 2025).


In [2]:
dataset = dataset.train_test_split(.1)

In [3]:
dataset['train'][0]

{'tibetan': 'རབ་དཀར་ཟླ་ཞུན་སྟོང་གི་ལང་ཚོ་ཅན།།',
 'phonetic': 'rabkar da zhün tong gi langtso chen',
 'english': 'Brilliant white with the vibrant vigor of a thousand moons'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [4]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google/switch-base-8"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Add Tibetan to Tokenizer

The T5 tokenizer does not notably support the Tibetan script. So, we need to add it manually. Once the characters have been added to the tokenizer, the model needs to have its token embeddings resized to accomodate the added tokens. This is all pretty straightforward, as seen in the code below.

In [5]:
# Generate a list of all Tibetan Unicode characters (U+0F00 to U+0FFF)
tibetan_chars = [chr(codepoint) for codepoint in range(0x0F00, 0x0FFF)]

# Add the Tibetan characters to the tokenizer's vocabulary
new_tokens = [char for char in tibetan_chars if char not in tokenizer.get_vocab()]

# Add new tokens to the tokenizer
tokenizer.add_tokens(new_tokens)

# Resize model embeddings to accommodate the new vocabulary size
model.resize_token_embeddings(len(tokenizer))

enc = tokenizer.encode(dataset['train'][0]['tibetan'])
dec = tokenizer.decode(enc)
dec


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


'རབ་དཀར་ཟླ་ཞུན་སྟོང་གི་ལང་ཚོ་ཅན།།</s>'

## Preprocess Data

The dataset can now be tokenized for training.

In [6]:
def translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['tibetan']]
    translation_targets = [example for example in examples['english']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return translation_model_inputs


In [7]:
def phonetic_translation_preprocess_function(examples):

    # Prepare translation inputs and targets
    translation_inputs = ['Translate Tibetan to English: ' + example for example in examples['phonetic']]
    translation_targets = [example for example in examples['english']]
    
    # Tokenize translation inputs and targets
    translation_model_inputs = tokenizer(translation_inputs, text_target=translation_targets, 
                                         max_length=300, truncation=True, padding="max_length")
    
    
    return translation_model_inputs

In [8]:
def transliteration_preprocess_function(examples):
    # Prepare transliteration inputs and targets
    transliteration_inputs = ['Transliterate: ' + example for example in examples['tibetan']]
    transliteration_targets = [example for example in examples['phonetic']]
    
    # Tokenize transliteration inputs and targets
    transliteration_model_inputs = tokenizer(transliteration_inputs, text_target=transliteration_targets, 
                                             max_length=300, truncation=True, padding="max_length")
    
    return transliteration_model_inputs

In [9]:
translation_tokenized_dataset = dataset.map(translation_preprocess_function, batched=True)

Map:   0%|          | 0/73103 [00:00<?, ? examples/s]

Map:   0%|          | 0/8123 [00:00<?, ? examples/s]

In [10]:
phonetic_translation_tokenized_dataset = dataset.map(phonetic_translation_preprocess_function, batched=True)

Map:   0%|          | 0/73103 [00:00<?, ? examples/s]

Map:   0%|          | 0/8123 [00:00<?, ? examples/s]

In [11]:
transliteration_tokenized_dataset = dataset.map(transliteration_preprocess_function, batched=True)

Map:   0%|          | 0/73103 [00:00<?, ? examples/s]

Map:   0%|          | 0/8123 [00:00<?, ? examples/s]

In [12]:
from datasets import concatenate_datasets

tokenized_dataset = {}

tokenized_dataset['train'] = concatenate_datasets([translation_tokenized_dataset['train'], phonetic_translation_tokenized_dataset['train'],transliteration_tokenized_dataset['train']])
tokenized_dataset['test'] = concatenate_datasets([translation_tokenized_dataset['test'], phonetic_translation_tokenized_dataset['test'], transliteration_tokenized_dataset['test']])

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor, EarlyStoppingCallback
from accelerate import Accelerator

accelerator = Accelerator()

early_stop = EarlyStoppingCallback(3)

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [14]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

Using the latest cached version of the module from /home/j/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--sacrebleu/28676bf65b4f88b276df566e48e603732d0b4afd237603ebdf92acaacf5be99b (last modified on Wed Jan 10 16:46:50 2024) since it couldn't be found locally at evaluate-metric--sacrebleu, or remotely on the Hugging Face Hub.


In [15]:
%env WANDB_PROJECT=huggingface

env: WANDB_PROJECT=huggingface


In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"thl-bo-en-tri-switch-v0",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=5
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset={'translation':translation_tokenized_dataset['test'], 'phonetic_translation': phonetic_translation_tokenized_dataset['test'],'transliteration':transliteration_tokenized_dataset['test']},
    processing_class=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Translation Loss,Translation Bleu,Translation Chrf,Translation Ter,Phonetic Translation Loss,Phonetic Translation Bleu,Phonetic Translation Chrf,Phonetic Translation Ter,Transliteration Loss,Transliteration Bleu,Transliteration Chrf,Transliteration Ter
1,0.086500,No log,0.108451,6.634500,19.837600,95.504700,0.105583,7.928900,21.738300,93.573800,0.014601,60.963500,84.174700,30.195500
2,0.063700,No log,0.082627,12.059900,27.068600,89.877300,0.080739,13.742300,28.822500,87.237000,0.009658,68.223000,85.916300,25.022700
3,0.052700,No log,0.069069,16.915100,32.191700,84.360600,0.067539,17.901600,33.359400,83.939200,0.008147,70.489100,86.336400,23.270800
4,0.044000,No log,0.061606,19.983900,34.913100,81.508200,0.060693,20.664100,35.752700,81.430900,0.007290,71.878200,86.590800,22.261700
5,0.044900,No log,0.059317,21.431700,36.439400,79.733300,0.058689,21.850000,36.780800,79.494600,0.007083,72.379300,86.636900,21.922100


TrainOutput(global_step=137070, training_loss=0.0790550078675689, metrics={'train_runtime': 107385.6909, 'train_samples_per_second': 10.211, 'train_steps_per_second': 1.276, 'total_flos': 1.173736378262016e+18, 'train_loss': 0.0790550078675689, 'epoch': 5.0})

## Quantize Model